**Modules**

To run this code, please ensure that you use python 3.10.11 and the tidy-finance environment installed:
https://www.tidy-finance.org/python/setting-up-your-environment.html

The code runs beautifully using python 3.13.5, but newer versions of python may not work, as pandas has a tendency to deprecate functions.

In [ ]:
# # Installs that my not be present in your environment
# !pip install Pathlib
# !pip install sqlite3
# !pip install re
# !pip install pandas-market-calendars

# Imports
import pandas as pd
import pandas_market_calendars as mcal
import numpy as np
import re

import sqlite3
import requests

import sys
import os
import io
import zipfile
from IPython.display import display
from pathlib import Path

from datetime import datetime

# Path to repository
sys.path.append("/Users/jonas/Documents/GitHub/Masters_Thesis_2025E")

# Import functions from helpers.py
from Project.Jonas.Models.helpers import convert_formats


Set start and end dates for preprocessing. This is important as ranking depends on the time period.

In [2]:
### Set start and end date
START_DATE = "2005-12-31"
END_DATE = "2021-12-31"

# Set epoch start
EPOCH_START = pd.Timestamp("1970-01-01") # Do not change

**Retrieving Data**

Download stock-characteristics from Xiu's website

In [3]:
# ### Retrieve data from Xiu's website ###

# # Download the ZIP file
# url = "https://dachxiu.chicagobooth.edu/download/datashare.zip"
# response = requests.get(url)

# with zipfile.ZipFile(io.BytesIO(response.content)) as z:
#     with z.open("datashare.csv") as f:
#         characteristics = pd.read_csv(f)

# # Save the CSV locally
# characteristics.to_csv("datashare.csv", index=False)

In [4]:
### If the file is already downloaded, read it directly from the local file system ###
path = "/Users/jonas/Library/CloudStorage/OneDrive-UniversityofCopenhagen/Kandidat/Thesis/2.0 Data/Factors Xiu/datashare.csv" # Update this path to where you saved the file

if os.path.exists(path):
    characteristics = pd.read_csv(path)
else:
    print("No path found")

# # Print the column names and formats to verify
# print(characteristics.dtypes)

Download CRSP data and macro from the SQLite database provided by Voigt

In [5]:
# ### Download and process tables from the SQLite database ###
# tidy_finance = sqlite3.connect(
#     database="/Users/jonas/Library/CloudStorage/OneDrive-UniversityofCopenhagen/Kandidat/Thesis/2.0 Data/SQLite/tidy_finance_r.sqlite"
# )

# # Read tables into pandas DataFrames
# tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", tidy_finance) 

# # # print the table names
# # print(tables)

# # Select tables to export
# selected_tables = ["crsp_monthly", "macro_predictors"]

# # Define absolute output folder path
# output_folder = "/Users/jonas/Library/CloudStorage/OneDrive-UniversityofCopenhagen/Kandidat/Thesis/2.0 Data/SQLite/Tables" # Update this path to your desired output folder

# # Create output folder if it doesn't exist
# os.makedirs(output_folder, exist_ok=True)

# # Export each table to parquet format
# for table_name in selected_tables:
#     df = pd.read_sql(f"SELECT * FROM {table_name};", tidy_finance)
#     df.to_parquet(os.path.join(output_folder, f"{table_name}.parquet"), index=False)
#     print(f"Exported: {table_name}")

# # Close the connection
# tidy_finance.close()

In [6]:
### If the tables are already downloaded and processed, read them directly from the local file system ###
path = "/Users/jonas/Library/CloudStorage/OneDrive-UniversityofCopenhagen/Kandidat/Thesis/2.0 Data/SQLite/Tables" # Update this path to where you saved the files

if os.path.exists(path):
    crsp_monthly = pd.read_parquet(os.path.join(path, "crsp_monthly.parquet"))
    macro_predictors = pd.read_parquet(os.path.join(path, "macro_predictors.parquet"))
else:
    print("No path found")
    
# # Print the column names and formats to verify
# print(crsp_monthly.dtypes)

Download linking tables from Ding

In [7]:
# ### Download and process linking tables from GitHub repository ###
# # Base URL for the GitHub repository
# base_url = "https://github.com/Wenzhi-Ding/Std_Security_Code/raw/main/other/"

# # List of linking tables to download
# linking_tables = [
#     "cik_gvkey.pq",
#     "cik_ticker.pq",
#     "gvkey_permco_permno.pq"
# ]

# # Local folder to store linking tables
# output_folder = "/Users/jonas/Library/CloudStorage/OneDrive-UniversityofCopenhagen/Kandidat/Thesis/2.0 Data/Linking Tables/Link to link" # Update this path to your desired output folder

# # Create output folder if it doesn't exist
# os.makedirs(output_folder, exist_ok=True)

# # Download, load, and export each file
# for link in linking_tables:
#     url = base_url + link
#     save_path = os.path.join(output_folder, link)
    
#     # Download .pq file
#     response = requests.get(url)
#     with open(save_path, "wb") as f:
#         f.write(response.content)
#     print(f"Downloaded: {link} -> {save_path}")

In [8]:
### If the linking tables are already downloaded and processed, read them directly from the local file system ###
path = "/Users/jonas/Library/CloudStorage/OneDrive-UniversityofCopenhagen/Kandidat/Thesis/2.0 Data/Linking Tables/Link to link" # Update this path to where you saved the files

if os.path.exists(path):
    cik_gvkey = convert_formats(pd.read_parquet(os.path.join(path, "cik_gvkey.pq"))) # Call convert_formats as there is misalignment between formats in the files - used here only as need for precision is not as great as for the others
    cik_ticker = convert_formats(pd.read_parquet(os.path.join(path, "cik_ticker.pq")))
    gvkey_permco_permno = convert_formats(pd.read_parquet(os.path.join(path, "gvkey_permco_permno.pq")))
else:
    print("No path found")

# # Print the column names and formats to verify
# print(cik_gvkey.dtypes)

Download insider trades from SEC

In [ ]:
# ### SEC Data Download and Preperation ###

# # Paths
# zip_path = Path(r"C:\Users\ufuky\OneDrive - University of Copenhagen\Jonas Theodor Østerby Schmidt's files - Kandidat\Thesis\2.0 Data\Insider Trading\zip")   # folder with ZIP files
# out_path = Path(r"C:\Users\ufuky\OneDrive - University of Copenhagen\Jonas Theodor Østerby Schmidt's files - Kandidat\Thesis\2.0 Data\Insider Trading\Insider Transaction Data 2021-2006")

# out_path.mkdir(parents=True, exist_ok=True)

# print(f"Looking for ZIP files in: {zip_path.resolve()}")

# # Loop through all ZIP files
# zip_files = list(zip_path.glob("*.zip"))
# print(f"Found {len(zip_files)} ZIP files")

# for zip_file in zip_files:
#     print(f"\nProcessing {zip_file.name}...")

#     try:
#         # Extract year + quarter from filename like "2010q1_form345"
#         match = re.search(r"(\d{4})q([1-4])", zip_file.stem.lower())
#         if not match:
#             print(f"Could not parse year/quarter from: {zip_file.stem}")
#             continue

#         year, quarter = match.groups()
#         quarter_code = f"{year[-2:]}Q{quarter}"  # e.g. 2010q1 -> 10Q1

#         with zipfile.ZipFile(zip_file, "r") as z:
#             members = z.namelist()
#             print(f"  Files inside ZIP: {members}")

#             for member in members:
#                 # Match the three relevant files
#                 if member.endswith("SUBMISSION.tsv"):
#                     new_name = f"{quarter_code}_SUBMISSION.tsv"
#                 elif member.endswith("REPORTINGOWNER.tsv"):
#                     new_name = f"{quarter_code}_REPORTINGOWNER.tsv"
#                 elif member.endswith("NONDERIV_TRANS.tsv"):
#                     new_name = f"{quarter_code}_NONDERIV_TRANS.tsv"
#                 else:
#                     continue

#                 # Extract file contents into a DataFrame
#                 with z.open(member) as f:
#                     df = pd.read_csv(f, sep="\t", low_memory=False)

#                 # Save with renamed file
#                 save_path = out_path / new_name
#                 df.to_csv(save_path, sep="\t", index=False)
#                 print(f" Saved {save_path.name} ({len(df)} rows)")

#     except Exception as e:
#         print(f"Error processing {zip_file.name}: {e}")
        
# base_path = Path(
#     r"C:\Users\ufuky\OneDrive - University of Copenhagen\Jonas Theodor Østerby Schmidt's files - Kandidat\Thesis\2.0 Data\Insider Trading\Insider Transaction Data 2021-2006"
# )

# # Columns to keep
# keep_cols = [
#     "ACCESSION_NUMBER",
#     "PERIOD_OF_REPORT",
#     "FILING_DATE",
#     "ISSUERCIK",
#     "ISSUERTRADINGSYMBOL",
#     "RPTOWNERCIK",
#     "RPTOWNERNAME",
#     "RPTOWNER_RELATIONSHIP",
#     "RPTOWNER_TITLE",
#     "RPTOWNER_RELATIONSHIP",
#     "RPTOWNER_STATE",
#     "SECURITY_TITLE",
#     "TRANS_DATE",
#     "TRANS_CODE",
#     "TRANS_ACQUIRED_DISP_CD",
#     "TRANS_SHARES",
#     "TRANS_PRICEPERSHARE",
#     "SHRS_OWND_FOLWNG_TRANS",
# ]

# def process_quarter(year, quarter):
#     """Load, merge, and clean insider trades for one quarter."""
#     try:
#         # Load files
#         df_sub = pd.read_csv(base_path / f"{str(year)[-2:]}Q{quarter}_SUBMISSION.tsv", sep="\t")
#         df_rep = pd.read_csv(base_path / f"{str(year)[-2:]}Q{quarter}_REPORTINGOWNER.tsv", sep="\t")
#         df_trans = pd.read_csv(base_path / f"{str(year)[-2:]}Q{quarter}_NONDERIV_TRANS.tsv", sep="\t")

#         # Merge
#         df_tmp = pd.merge(df_sub, df_rep, on="ACCESSION_NUMBER", how="inner")
#         df_merged = pd.merge(df_tmp, df_trans, on="ACCESSION_NUMBER", how="inner")

#         # Keep only selected columns
#         df_clean = df_merged[df_merged.columns.intersection(keep_cols)].copy()

#         # Add transaction value
#         df_clean["TOTAL"] = (
#             pd.to_numeric(df_clean["TRANS_SHARES"], errors="coerce") *
#             pd.to_numeric(df_clean["TRANS_PRICEPERSHARE"], errors="coerce")
#         )

#         # Filter by P and S only
#         df_clean = df_clean[df_clean["TRANS_CODE"].isin(["P", "S"])]

#         # Remove mixed P/S accessions
#         check_codes = (
#             df_clean.groupby("ACCESSION_NUMBER")["TRANS_CODE"]
#             .nunique()
#             .reset_index(name="unique_trans_codes")
#         )
#         bad_accessions = check_codes[check_codes["unique_trans_codes"] > 1]["ACCESSION_NUMBER"]

#         df_uniform = df_clean[~df_clean["ACCESSION_NUMBER"].isin(bad_accessions)].copy()

#         print(f"{year}Q{quarter}: {len(df_uniform)} rows kept")
#         return df_uniform

#     except FileNotFoundError:
#         print(f"Missing file for {year}Q{quarter}")
#         return pd.DataFrame()

# # Collect everything into one big DataFrame
# dfs_all = []

# for year in range(2006, 2022):  # inclusive 2021
#     for quarter in range(1, 5):
#         df_q = process_quarter(year, quarter)
#         if not df_q.empty:
#             dfs_all.append(df_q)

# # Save one combined file
# if dfs_all:
#     df_all = pd.concat(dfs_all, ignore_index=True)
#     out_file_all = base_path / "trades_2006-2021.txt"
#     df_all.to_csv(out_file_all, sep="\t", index=False)
#     print(f"Saved {len(df_all)} total rows across 2006–2021 → {out_file_all}")

In [9]:
### The insider trades have been downloaded, see seperate file ###
path = "/Users/jonas/Library/CloudStorage/OneDrive-UniversityofCopenhagen/Kandidat/Thesis/2.0 Data/Insider Trading/Insider Transaction Data 2021-2006/trades_2006-2021.txt"

if os.path.exists(path):
    insider_trades = pd.read_csv(path, sep="\t")
else: 
    print("No path found")
    
# # Print the column names and formats to verify
# print(insider_trades.dtypes)

**Preparing GKX**

Prepare stock characteristics

In [10]:
### Prepare stock characteristics ###

# Print the first couple of rows
# # print(characteristics.head())

# Rename DATE to month
characteristics = characteristics.rename(columns={"DATE": "month"})

# Convert to datetime
characteristics["month"] = (
    pd.to_datetime(characteristics["month"].astype(str), format="%Y%m%d", errors="coerce")
      .dt.to_period("M")
      .dt.to_timestamp("M")
)

# Change period
characteristics = characteristics[(characteristics["month"] >= pd.to_datetime(START_DATE)) & (characteristics["month"] <= pd.to_datetime(END_DATE))]
        
# For each characteristic change name to char_value
characteristics = characteristics.rename(columns=lambda x: "char_" + x if x not in ["permno", "month", "sic2"] else x)

# Function to impute missing sic2 values
def impute_sic2(df):
    
    # Create a copy to avoid modifying the original DataFrame
    df = df.copy()
    
    # Sort by permno and month
    df = df.sort_values(["permno", "month"])

    # Backfill sic2 within each permno
    df["sic2_bfill"] = df.groupby("permno")["sic2"].bfill()

    # Compute sic2 mode per permno (or 0 if all missing)
    sic2_final = (
        df.groupby("permno")["sic2_bfill"]
        .agg(lambda x: x.mode().iloc[0] if not x.dropna().empty else 0)
    )

    # Fill all missing sic2 values using the per-permno mapping
    df["sic2"] = df["sic2"].fillna(df["permno"].map(sic2_final))

    # Clean up helper column
    df = df.drop(columns=["sic2_bfill"])

    return df

# Apply the imputation function 
characteristics = impute_sic2(characteristics)

# # Display the first couple of rows
# display(characteristics.head())

Prepare CRSP monthly data

In [11]:
### Prepare CRSP monthly data ###

# # Print the first couple of rows
# print(crsp_monthly.head())

# Rename date to month
crsp_monthly = crsp_monthly.rename(columns={"date": "month"})

# Convert date type, and set to month to months end
crsp_monthly["month"] = EPOCH_START + pd.to_timedelta(crsp_monthly["month"], unit="D")
crsp_monthly["month"] = crsp_monthly["month"].dt.to_period("M").dt.to_timestamp("M")

# Change period
crsp_monthly = crsp_monthly[(crsp_monthly["month"] >= pd.to_datetime(START_DATE)) & (crsp_monthly["month"] <= pd.to_datetime(END_DATE))]

# Only use selected columns
crsp_monthly = crsp_monthly[["month", "permno", "ret_excess", "prc", "shrout", "mktcap", "mktcap_lag"]]

# # Display the first couple of rows
# display(crsp_monthly.head())

Prepare macropredictors by Goyal.

Download raw predictors via: https://docs.google.com/spreadsheets/d/1bM7vCWd3WOt95Sf9qjLPZjoiafgF_8EG.

In [12]:
### Prepare macropredictors data ###

# # Print the first couple of rows
# print(macropredictors.head())

# Rename date to month
macro_predictors = macro_predictors.rename(columns={"date": "month"})

# Convert date type
macro_predictors["month"] = EPOCH_START + pd.to_timedelta(macro_predictors["month"], unit="D")
macro_predictors["month"] = macro_predictors["month"].dt.to_period("M").dt.to_timestamp("M")

# Lag one month
macro_predictors["month"] = macro_predictors["month"] + pd.offsets.MonthEnd(1)

# Change period
macro_predictors = macro_predictors[(macro_predictors["month"] >= pd.to_datetime(START_DATE)) & (macro_predictors["month"] <= pd.to_datetime(END_DATE))]

# Only use selected columns
macro_predictors = macro_predictors[["month", "dp", "ep", "bm", "ntis", "tbl", "tms", "dfy", "svar"]]

# For each predictor change name to macro_predictor
macro_predictors = macro_predictors.rename(columns=lambda x: f"macro_{x}" if x != "month" else x)

# # Display the first couple of rows
# display(macro_predictors.head())

Link tables: cik → permno

In [13]:
### Get all cik_gvkeys where sec_end_date is larger than December 31, 2005 and sec_start_date is smaller than January 31, 2021 ###

# Ensure datetime format for sec_start_date and sec_end_date
cik_gvkey["sec_start_date"] = (pd.to_datetime(cik_gvkey["sec_start_date"], format="%Y%m%d", errors="coerce"))
cik_gvkey["sec_end_date"] = (pd.to_datetime(cik_gvkey["sec_end_date"], format="%Y%m%d", errors="coerce"))

# Print "Not all sec_x_date have values" if there are any NA values in sec_start_date or sec_end_date
assert cik_gvkey["sec_start_date"].isna().sum() == 0, "Not all sec_start_date have values"
assert cik_gvkey["sec_end_date"].isna().sum() == 0, "Not all sec_end_date have values"

# Filter cik_gvkey for sec_end_date larger than December 31, 2005 and sec_start_date smaller than January 31, 2021
cik_gvkey = cik_gvkey[cik_gvkey["sec_start_date"] <= pd.to_datetime(END_DATE)]
cik_gvkey = cik_gvkey[cik_gvkey["sec_end_date"] >= pd.to_datetime(START_DATE)].reset_index(drop=True)

### Get all gvkey_permco_permno where linkdt is smaller than January 31, 2021 - Issues with linkenddt  ###

# Ensure datetime format for linkdt
gvkey_permco_permno["linkdt"] = pd.to_datetime(
    gvkey_permco_permno["linkdt"].astype("Int64").astype(str),
    format="%Y%m%d",
    errors="coerce"
)

# Print if there are any NA values in linkdt
assert gvkey_permco_permno["linkdt"].isna().sum() == 0, "Not all linkdt have values"

# Filter gvkey_permco_permno for linkdt smaller than January 31, 2021
gvkey_permco_permno = gvkey_permco_permno[gvkey_permco_permno["linkdt"] <= pd.to_datetime(END_DATE)].reset_index(drop=True)

### Merge cik_gvkey with gvkey_permco_permno to get cik_permno ###

# Merge on gvkey and keep only relevant columns
cik_permno_linking_table = (
    pd.merge(cik_gvkey, gvkey_permco_permno, on="gvkey", how="inner")
      .loc[:, ["cik", "lpermno"]]
      .rename(columns={"lpermno": "permno"})
      .dropna(subset=["permno"])
      .drop_duplicates()
      .reset_index(drop=True)
)

### Remove permnos with multiple ciks and vice versa to ensure one-to-one mapping ###

# Count unique ciks per permno and vice versa
unique_cik_per_permno = cik_permno_linking_table.groupby("permno")["cik"].nunique()
unique_permno_per_cik = cik_permno_linking_table.groupby("cik")["permno"].nunique()

# Find ciks and permnos where there are multiple ciks per permno and vice versa
ambiguous_permnos = unique_cik_per_permno[unique_cik_per_permno > 1].index
ambiguous_ciks = unique_permno_per_cik[unique_permno_per_cik > 1].index

# Filter out ambiguous ciks and permnos
cik_permno_linking_table = cik_permno_linking_table[~cik_permno_linking_table["permno"].isin(ambiguous_permnos)]
cik_permno_linking_table = cik_permno_linking_table[~cik_permno_linking_table["cik"].isin(ambiguous_ciks)]

# Reset index of cik_permno_linking_table
cik_permno_linking_table = cik_permno_linking_table.reset_index(drop=True)

# # Finalized cik_permno linking table
# display(cik_permno_linking_table.head())

In [14]:
# Print report
print(f"Final cik_permno linking table has {cik_permno_linking_table.shape[0]} unique cik_permno pairs.")

Final cik_permno linking table has 12977 unique cik_permno pairs.


**Merging GKX**

In [15]:
### Convert permno for characteristics and crsp_monthly to the same format ###
# characteristics
characteristics["permno"] = characteristics["permno"].astype(str).str.lstrip("0")
characteristics["permno"] = pd.to_numeric(characteristics["permno"], errors="coerce").astype("Int32") # Use "Int32" to allow for NA values

# crsp_monthly
crsp_monthly["permno"] = crsp_monthly["permno"].astype(str).str.lstrip("0")
crsp_monthly["permno"] = pd.to_numeric(crsp_monthly["permno"], errors="coerce").astype("Int32")

### Merge all data ###
# Merge characteristics with crsp_monthly on month and permno
merged_df = pd.merge(characteristics, crsp_monthly, on=["month", "permno"], how="inner")

# Merge the result with cik_permno on permno to get cik
merged_df = pd.merge(merged_df, cik_permno_linking_table, on="permno", how="inner")

# Merge the merged_df with macro_predictors on month
merged_df = pd.merge(merged_df, macro_predictors, on="month", how="inner")

# Rearrange columns
cols = merged_df.columns.tolist()
cols = ["month", "cik", "permno", "ret_excess", "prc", "shrout", "mktcap", "mktcap_lag"] + [col for col in cols if col not in ["month", "cik", "permno", "ret_excess", "prc", "shrout", "mktcap", "mktcap_lag"]]
merged_df = merged_df[cols].reset_index(drop=True)

# # Display the first couple of rows
# display(merged_df.head())

In [16]:
### Create stock universe for breakpoints ###
stock_universe = merged_df[["month", "cik", "permno", "ret_excess", "prc", "shrout", "mktcap", "mktcap_lag"]].drop_duplicates().reset_index(drop=True)

# Save as stock_universe.csv
stock_universe.to_csv("/Users/jonas/Library/CloudStorage/OneDrive-UniversityofCopenhagen/Kandidat/Thesis/2.0 Data/Stock Universe/stock_universe.csv", index=False)

In [ ]:
### Create cik_permno linking table with only cik and permno from merged_df. Why? Because insider trades have to be linked to cik and permno with returns data later on. ###
insider_cik_permno_linking_table = merged_df[["month", "cik", "permno", "mktcap"]].drop_duplicates().reset_index(drop=True)

# Print for later use 
print(f"No. of unique ciks: {insider_cik_permno_linking_table.cik.nunique()}, No. of unique permnos: {insider_cik_permno_linking_table.permno.nunique()}")

No. of unique ciks: 6760, No. of unique permnos: 6760


**Preparing insider trading data**

In [ ]:
### The insider trades have been downloaded, see seperate file ###
path = "/Users/jonas/Library/CloudStorage/OneDrive-UniversityofCopenhagen/Kandidat/Thesis/2.0 Data/Insider Trading/Insider Transaction Data 2021-2006/trades_2006-2021.txt"

if os.path.exists(path):
    insider_trades = pd.read_csv(path, sep="\t")
else: 
    print("No path found")
    
# # Print the column names and formats to verify
# print(insider_trades.dtypes)

### Prepare insider trades data ###
insider_trades.columns = insider_trades.columns.str.lower()

# # Print list of column names
# print(insider_trades.columns.tolist())

# Drop columns that are not needed for the analysis
cols_to_keep = ['filing_date', 
                'issuercik', 
                'issuertradingsymbol', 
                'rptownercik', 
                'rptownername', 
                'rptowner_relationship', 
                'rptowner_title',
                'security_title', 
                'trans_date', 
                'trans_code', 
                'trans_shares', 
                'trans_pricepershare', 
                'total']

insider_trades = insider_trades[cols_to_keep]

# Rename columns for clarity
insider_trades = insider_trades.rename(columns={
    "issuercik": "cik",
    "issuertradingsymbol": "ticker",
    "rptownercik": "reporting_cik",
    "rptownername": "name",
    "rptowner_relationship": "relationship",
    "rptowner_title": "officer_title",
    "security_title": "security",
    "trans_date": "transaction_date",
    "trans_code": "transaction_type",
    "trans_shares": "shares",
    "trans_pricepershare": "price_per_share",
    "total": "total",
})

# # Print names and formats
# print(insider_trades.dtypes)

# Print len of insider_trades
print(f"No. of insider trades: {len(insider_trades)}")

No. of insider trades: 3754776


In [ ]:
### Dates ###

# Columns to Datetime
cols_to_dt = ["filing_date", "transaction_date"]

# Convert using the appropriate datetime format
insider_trades[cols_to_dt] = insider_trades[cols_to_dt].apply(
    lambda x: pd.to_datetime(x, format="%d-%b-%Y", errors="coerce").dt.normalize()
)

# Filter insider trades to be within the analysis period
insider_trades = insider_trades[
    insider_trades[cols_to_dt].apply(lambda x: x.between((pd.to_datetime(START_DATE)+pd.to_timedelta(1, unit="D")), pd.to_datetime(END_DATE))).all(axis=1)
]

### Set condition ###
outsider = True # True if outsider trades, False if insider trades

# Filter based on outsider or insider trades
if outsider:
    # Remove rows where filing_date and transaction_date are not the same month
    insider_trades = insider_trades[
        insider_trades["filing_date"].dt.to_period("M") == insider_trades["transaction_date"].dt.to_period("M")
    ].reset_index(drop=True)
else:
    # Keep all insider trades
    pass

# Create a month column based on transaction_date
insider_trades["month"] = insider_trades["transaction_date"].dt.to_period("M").dt.to_timestamp("M")
    
# Print len of insider_trades
print(f"No. of insider trades: {len(insider_trades)}")

No. of insider trades: 3367525


With outsider == False, 3,742,752 transactions. 


With outsider == True, 3,367,525 transactions.

In [ ]:
### Generate list of market days (NYSE trading days) for clustering ###
market_cal = mcal.get_calendar("NYSE")

# Get the NYSE schedule (open/close times)
schedule = market_cal.schedule(start_date=START_DATE, end_date=END_DATE)

# Convert schedule into a list of actual trading days
market_days = mcal.date_range(schedule, frequency="1D")

# Ensure the result is a clean DatetimeIndex without time-zone and normalized to midnight
market_days = pd.to_datetime(market_days).tz_localize(None).normalize()

# Assert market dates are monotoniccally increasing
assert market_days.is_monotonic_increasing, "Market days are not monotonically increasing"

In [ ]:
### Print number of transactions ###
print(f"No. of insider trades: {len(insider_trades)}")

# Check for any transaction dates that are not market days
invalid_dates = insider_trades[~insider_trades["transaction_date"].isin(market_days)]

# Print the number of invalid dates found
print(f"Number of transactions with invalid transaction dates: {len(invalid_dates)}")

# As 1000 rows are negligible compared to the full sample, they are dropped
insider_trades = insider_trades[insider_trades["transaction_date"].isin(market_days)].reset_index(drop=True)

# Print number of transactions after removing invalid dates
print(f"No. of insider trades after removing invalid dates: {len(insider_trades)}")

No. of insider trades: 3367525
Number of transactions with invalid transaction dates: 2929
No. of insider trades after removing invalid dates: 3364596


In [ ]:
### Merge with cik_permno linking table to get permno ###

# Merge insider trades with cik_permno linking table on cik and month
insider_trades = insider_trades.merge(
    insider_cik_permno_linking_table,
    on=["cik", "month"],
    how="inner"
).sort_values(by=["cik", "transaction_date"]).reset_index(drop=True)

# Drop month column as it is no longer needed
insider_trades = insider_trades.drop(columns=["month"])

# Set permno after cik for easier viewing
cols_order = insider_trades.columns.tolist()
cols = ["cik", "permno"] + [col for col in cols_order if col not in ["cik", "permno"]]
insider_trades = insider_trades[cols]

# Print len of insider_trades
print(f"No. of insider trades: {len(insider_trades)}")

No. of insider trades: 2201319


In [ ]:
### Cleaning ###

# Rows before cleaning
before = len(insider_trades)
print(f"Rows before cleaning: {before:,}\n")

# Copy df
df0 = insider_trades.copy()

# Valid ticker
df0["ticker"] = df0["ticker"].astype(str).str.upper() # Ensure ticker is uppercase string for consistent filtering
mask0 = ~df0["ticker"].isin(["NONE", "N/A", "NA"])
print(f"Removed by invalid ticker: {len(df0) - mask0.sum():,}")
df0 = df0[mask0]

# total > 10_000 and not missing
mask1 = df0["total"].notna() & (df0["total"] > 10_000)
print(f"Removed by invalid/missing total: {len(df0) - mask1.sum():,}")
df1 = df0[mask1].copy()

# At least 100 shares
mask2 = (df1["shares"].notna() & (df1["shares"] >= 100))
print(f"Removed by small/missing trades (<100 shares): {len(df1) - mask2.sum():,}")
df2 = df1[mask2]

# Market cap filter (5% of firm value)
scale = 1_000_000  # market cap is in millions
mask3 = df2["mktcap"].notna() & (df2["total"] <= 0.05 * (df2["mktcap"] * scale))
print(f"Removed by market cap filter (>5% of firm value): {len(df2) - mask3.sum():,}")
df3 = df2[mask3]

# Remove exact duplicates
before_dupes = len(df3)
df3 = df3.drop_duplicates()
after_dupes = len(df3)
print(f"Removed exact duplicates: {before_dupes - after_dupes:,}")

# Remove duplicates where only name/reporting_cik differs
if "name" in df3.columns and "reporting_cik" in df3.columns:
    before_near_dupes = len(df3)
    df3 = df3.drop_duplicates(
        subset=[col for col in df3.columns if col not in ["name", "reporting_cik"]]
    )
    after_near_dupes = len(df3)
    print(f"Removed near-duplicates (name/reporting_cik differences): {before_near_dupes - after_near_dupes:,}")

# Drop mktcap column as it is no longer needed
df3 = df3.drop(columns=["mktcap"])

# Rows after cleaning
after = len(df3)
print(f"\nRows after cleaning: {after:,}")
print(f"Total removed: {before - after:,}")

# Final cleaned DataFrame
insider_trades = df3

Rows before cleaning: 2,201,319

Removed by invalid ticker: 165
Removed by invalid/missing total: 696,785
Removed by small/missing trades (<100 shares): 3,782
Removed by market cap filter (>5% of firm value): 10,668
Removed exact duplicates: 26,535
Removed near-duplicates (name/reporting_cik differences): 249,316

Rows after cleaning: 1,214,068
Total removed: 987,251


Insider = 1,313,093

Outsider = 1,214,068

In [ ]:
### Change Titles ### 

# Abbreviation normalization map
_ABBREV_MAP = {
    r"\b&\b": " and ",
    r"\bevps?\b": " executive vice president ",
    r"\bsvp\b": " senior vice president ",
    r"\bsr\.\b": " senior ",
    r"\bsr\b": " senior ",
    r"\bavp\b": " assistant vice president ",
    r"\bvp\b": " vice president ",
    r"\bceo\b": " chief executive officer ",
    r"\bcfo\b": " chief financial officer ",
    r"\bcoo\b": " chief operating officer ",
    r"\bcto\b": " chief technology officer ",
}
_STOP_PUNCT = r"[^a-z0-9/&\- ]"

# Normalization function
def normalize_title(s: str) -> str:
    if pd.isna(s):
        return None
    t = str(s).lower().strip()
    t = re.sub(_STOP_PUNCT, " ", t)
    for pat, repl in _ABBREV_MAP.items():
        t = re.sub(pat, repl, t)
    t = t.replace("/", " ").replace("-", " ")
    t = re.sub(r"\s+", " ", t).strip()
    return t

### Change below to all lower.string? 

# Categorization rules (updated hierarchy)
def categorize_simple(t: str) -> str:
    if t is None:
        return None

    # CEO
    if "chief executive officer" in t or "ceo" in t:
        return "ceo"
    # CFO
    elif any(w in t for w in ["chief financial officer", "cfo", "treasurer", "controller", "finance"]):
        return "cfo"
    # COO
    elif any(w in t for w in ["chief operating officer", "coo", "operations"]):
        return "coo"
    # Other Officers (all other "Chiefs" such as CTO, CLO, etc.)
    elif any(w in t for w in [
        "chief", "cto", "legal", "marketing", "information", "technology",
        "strategy", "compliance", "administration", "hr"
    ]):
        return "other_officer"
    # Vice Presidents
    elif any(w in t for w in ["vice president", "executive vice president", "senior vice president"]):
        return "vice_president"
    # Directors / Board members
    elif any(w in t for w in ["director", "board", "chairman", "managing director"]):
        return "director"
    # Owners (≥10%)
    elif "owner" in t:
        return "ten_percent_owner"
    # Default fallback
    else:
        return "other_officer"

# Apply normalization and categorization
insider_trades["title"] = insider_trades["officer_title"].apply(normalize_title)
insider_trades["title"] = insider_trades["title"].apply(categorize_simple)

# Backfill from relationship if available
if "relationship" in insider_trades.columns: 
    insider_trades["title"] = insider_trades["title"].fillna(
        insider_trades["relationship"].apply(normalize_title).apply(categorize_simple)
    )

# Collapse into 7 categories (enforcing clean labels)
VALID_CATS = ["ceo", "cfo", "coo", "other_officer", "vice_president", "director", "ten_percent_owner"]

def enforce_categories(s: str) -> str:
    if pd.isna(s):
        return "other_officer"
    if s in VALID_CATS:
        return s
    if "vice" in s.lower():
        return "vice_president"
    if "director" in s.lower():
        return "director"
    if "owner" in s.lower():
        return "ten_percent_owner"
    return "other_officer"

insider_trades["title"] = insider_trades["title"].apply(enforce_categories)

# Move title next to relationship if it exists
if "relationship" in insider_trades.columns:
    cols = list(insider_trades.columns)
    rel_idx = cols.index("relationship")
    cols.insert(rel_idx + 1, cols.pop(cols.index("title")))
    insider_trades = insider_trades[cols]

print("Unique categories:", insider_trades["title"].unique())
print(insider_trades["title"].value_counts())

Unique categories: ['other_officer' 'vice_president' 'cfo' 'ceo' 'coo' 'director'
 'ten_percent_owner']
title
other_officer        493961
ceo                  246018
vice_president       163495
ten_percent_owner    130914
cfo                   88898
coo                   49685
director              41097
Name: count, dtype: int64


**Feature Engineering**

In [ ]:
### Dummy Variables for Officer Titles ###
# Create dummies 
insider_trades = pd.get_dummies(
    insider_trades,
    columns=["title"],
    prefix="is_tit",       
    drop_first=False,      # Keep all categories
    dtype="float32",
    sparse=False
)

# # Print column names and types
# print(insider_trades.dtypes)

### Dummy Variables for Transaction Type ###
# Standardize transaction types
insider_trades["transaction_type"] = (
    insider_trades["transaction_type"]
    .astype(str)
    .str.strip()
    .str.lower() # Ensure consistent casing
)

# Create dummies
insider_trades = pd.get_dummies(insider_trades,
                                columns=["transaction_type"],
                                prefix="is_txn",         
                                drop_first=False,       # Keep all categories
                                dtype="float32",
                                sparse=False
)

# Rename for readability
insider_trades = insider_trades.rename(columns={
    "is_txn_p": "is_txn_purchase",
    "is_txn_s": "is_txn_sell",
})

# # Print column names and types
# print(insider_trades.dtypes)

In [ ]:
### Interaction is_purchase and is_sell with officer titles ###
# List of officer title dummies
officer_titles = [col for col in insider_trades.columns if col.startswith("is_tit_")]

# For loop
for txn in ["is_txn_purchase", "is_txn_sell"]:
    for title in officer_titles:
        col_name = f"{txn}_x_{title}"
        insider_trades[col_name] = (
            insider_trades[txn] * insider_trades[title]
            ).astype("float32")
        
# Drop titles from dataframe
insider_trades = insider_trades.drop(columns=officer_titles)

# # Print column names and types
# print(insider_trades.dtypes)

In [ ]:
### Construct is_vol_purchase and is_vol_sell ###
insider_trades["is_vol_purchase"] = (insider_trades["is_txn_purchase"] * insider_trades["total"]).astype("float32")
insider_trades["is_vol_sell"] = (insider_trades["is_txn_sell"] * insider_trades["total"]).astype("float32")

In [ ]:
# ### Function to Construct Cluster Features: Detect cluster episodes for one firm and one transaction type, returns column with 1 on the LAST day of each cluster episode ###
def clusters_per_type(group, txn_col):

    # Create a copy of the group to avoid modifying the original DataFrame
    group = group.copy()
    
    # Ensure datetime format for transaction_date
    group["transaction_date"] = pd.to_datetime(group["transaction_date"]).dt.normalize()
    
    # Initialize cluster column to 0
    group[f"{txn_col}_cluster"] = 0

    # Filter to transaction true
    txn_true = group[group[txn_col] == 1].copy()

    # If no trades of this type -> no clusters
    if txn_true.empty:
        return group

    # Sort by transaction date
    txn_true = txn_true.sort_values("transaction_date")

    # Extract unique dates
    dates = (
        txn_true["transaction_date"]
        .drop_duplicates()
        .sort_values()
        .reset_index(drop=True)
    )
    
    # Ensure that len(dates) >= 2 for cluster detection
    if len(dates) < 2:
        return group

    # Find all cluster-participating days
    cluster_days = []

    for i in range(len(dates) - 1):
        d1, d2 = dates[i], dates[i + 1]

        # Must be consecutive market days
        if d2 == market_days[market_days.get_loc(d1) + 1]:

            # Count unique insiders over d1+d2
            insiders = (
                txn_true.loc[txn_true["transaction_date"].isin([d1, d2]), "reporting_cik"]
                .nunique()
            )

            if insiders >= 2:
                cluster_days.append(d1)
                cluster_days.append(d2)

    # If no cluster days found return group
    if not cluster_days:
        return group

    # Collapse cluster-days into cluster episodes
    cluster_days = pd.Series(sorted(set(cluster_days)))

    # Convert cluster_days to market-day index positions
    market_days_idx = pd.Index(market_days)
    cluster_pos = cluster_days.map(market_days_idx.get_loc)

    # Identify new blocks where the gap is > 1 market day
    blocks = cluster_pos.diff().gt(1).cumsum()

    # The END of each block is the last cluster day in that block
    cluster_ends = cluster_days.groupby(blocks).max()

    # Mark the last row per cluster episode
    for end_day in cluster_ends:
        
        # All trade rows on that day of this transaction
        rows_on_day = txn_true[txn_true["transaction_date"] == end_day]

        # If multiple rows, use the last one
        if not rows_on_day.empty:
            last_idx = rows_on_day.index[-1]
            group.loc[last_idx, f"{txn_col}_cluster"] = 1

    return group

### Apply cluster logic to entire dataset ###
def compute_purchase_sell_clusters(df):

    # Copy to avoid modifying original dataframe
    df = df.copy()
    
    # Sort by cik and transaction_date
    df = df.sort_values(["cik", "transaction_date"])

    # Purchase clusters
    df = (
        df.groupby("cik", group_keys=False)
          .apply(lambda x: clusters_per_type(x, "is_txn_purchase"))
    )

    # sell clusters
    df = (
        df.groupby("cik", group_keys=False)
          .apply(lambda x: clusters_per_type(x, "is_txn_sell"))
    )

    return df

### Call Function to Compute Clusters ###
insider_trades = compute_purchase_sell_clusters(insider_trades) # Ensure no pandas version > 2.2.3 - groupby issues may arise for later versions (deprecation of group_keys)

# Ensure cluster columns are float32
insider_trades["is_txn_purchase_cluster"] = insider_trades["is_txn_purchase_cluster"].astype("float32")
insider_trades["is_txn_sell_cluster"] = insider_trades["is_txn_sell_cluster"].astype("float32")

/var/folders/k3/9jyk10k50tg1bg0x762zj6200000gn/T/ipykernel_41890/962994462.py:96: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: clusters_per_type(x, "is_txn_purchase"))
/var/folders/k3/9jyk10k50tg1bg0x762zj6200000gn/T/ipykernel_41890/962994462.py:102: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: clusters_per_type(x, "is_txn_sell"))


In [ ]:
### Classify Routine vs. Opportunistic Insiders ###
def classify_routine_opportunistic_monthly(df_trades_clean, min_year=2009):
    """
    Classify insiders as routine or opportunistic (CMP 2012) based on transaction_date.
    """

    # Copy to avoid modifying original dataframe
    df = df_trades_clean.copy()
    
    # Ensure transaction_date is datetime
    df["transaction_date"] = pd.to_datetime(df["transaction_date"]).dt.normalize()

    # Dates
    df["year"] = df["transaction_date"].dt.year
    df["month"] = df["transaction_date"].dt.month
    df["month_period"] = df["transaction_date"].dt.to_period("M")

    # Find routine insiders (3-year same-month pattern)
    months_per_year = (
        df.groupby(["reporting_cik", "year"])["month"]
          .agg(lambda x: set(x))
          .reset_index()
    )

    # Find start year of routine trader
    routine_start = {}
    for insider, sub in months_per_year.groupby("reporting_cik"):
        sub = sub.sort_values("year")
        years = sub["year"].to_numpy()
        months = sub["month"].to_list()

        for j in range(len(years) - 2):
            y1, y2, y3 = years[j], years[j+1], years[j+2]

            # Consecutive years
            if y2 == y1 + 1 and y3 == y1 + 2:
                common = months[j] & months[j+1] & months[j+2]
                if common:
                    # Routine starts in year T+1
                    routine_start[insider] = y3 + 1
                    break

    # Create DataFrame of routine_starts
    df_routine = pd.DataFrame(
        list(routine_start.items()),
        columns=["reporting_cik", "routine_start_year"]
    )

    df = df.merge(df_routine, on="reporting_cik", how="left")

    # Classify trades
    df["trader_type"] = np.where(
        df["routine_start_year"].notna() &
        (df["year"] >= df["routine_start_year"]),
        "routine",
        "opportunistic"
    )

    # Optional trimming
    if min_year is not None:
        df = df[df["year"] >= min_year].copy()

    # Get dummies for trader type and transaction type
    df["is_opp_buy"]  = ((df["trader_type"] == "opportunistic") & df["is_txn_purchase"]).astype("float32")
    df["is_opp_sell"] = ((df["trader_type"] == "opportunistic") & df["is_txn_sell"]).astype("float32")
    df["is_rtn_buy"]  = ((df["trader_type"] == "routine")       & df["is_txn_purchase"]).astype("float32")
    df["is_rtn_sell"] = ((df["trader_type"] == "routine")       & df["is_txn_sell"]).astype("float32")

    # Aggregate to firm–month (only months with trades)
    monthly = (
        df.groupby(["cik", "month_period"])[
            ["is_opp_buy", "is_opp_sell", "is_rtn_buy", "is_rtn_sell"]
        ]
        .max()
        .reset_index()
        .rename(columns={"month_period": "month"})
    )

    return monthly[["cik", "month", "is_opp_buy", "is_opp_sell", "is_rtn_buy", "is_rtn_sell"]]

### Call Function to Classify Routine vs. Opportunistic Insiders ###
insider_trader_types_monthly = classify_routine_opportunistic_monthly(insider_trades, min_year=2009)

### Ensure month is datetime and end of month ###

# Set month to timestamp
insider_trader_types_monthly["month"] = insider_trader_types_monthly["month"].dt.to_timestamp("M")

# Ensure that period is within analysis window
insider_trader_types_monthly = insider_trader_types_monthly[
    (insider_trader_types_monthly["month"] >= pd.to_datetime(START_DATE)) &
    (insider_trader_types_monthly["month"] <= pd.to_datetime(END_DATE))
].reset_index(drop=True)

# Print report
print("Insider Trader Types Monthly Summary:")
print(insider_trader_types_monthly[["is_opp_buy", "is_opp_sell", "is_rtn_buy", "is_rtn_sell"]].sum())

Insider Trader Types Monthly Summary:
is_opp_buy     29580.0
is_opp_sell    99349.0
is_rtn_buy      3978.0
is_rtn_sell    29913.0
dtype: float32


In [ ]:
### Function to Classify Insider Silence Events (PPN, SSN): Return Only firm-month where an event occurs.  ###
def classify_insider_silence_fast(df):
    """
    Classify insider silence events (PPN, SSN) based on transaction_date.
    PPN: purchase-purchase-no trade
    SSN: sell-sell-no trade"""

    # Copy data to avoid modifying the original
    df = df.copy()

    # Ensure transaction_date is datetime
    df["transaction_date"] = pd.to_datetime(df["transaction_date"]).dt.normalize()

    # Prepare date columns
    df["year"] = df["transaction_date"].dt.year
    df["month"] = df["transaction_date"].dt.month

    # Monthly insider activity
    g = (
        df.groupby(["cik", "reporting_cik", "year", "month"])[["is_txn_purchase", "is_txn_sell"]]
          .any()
          .reset_index()
    )
    g["has_any"] = g["is_txn_purchase"] | g["is_txn_sell"]
    
    # Months where insiders bought/sold
    purchase = g[g["is_txn_purchase"]][["cik", "reporting_cik", "month", "year"]]
    sell = g[g["is_txn_sell"]][["cik", "reporting_cik", "month", "year"]]

    # Build T-year candidates (PPN purchases)
    purchase_tminus2 = purchase.assign(year=lambda x: x["year"] + 2)
    purchase_tminus1 = purchase.assign(year=lambda x: x["year"] + 1)

    cand_ppn_T = (
        purchase_tminus2.merge(
            purchase_tminus1,
            on=["cik", "reporting_cik", "month", "year"],
            how="inner"
        )
        [["cik", "reporting_cik", "month", "year"]]
        .drop_duplicates()
    )

    # Build T-year candidates (SSN sells)
    sell_tminus2 = sell.assign(year=lambda x: x["year"] + 2)
    sell_tminus1 = sell.assign(year=lambda x: x["year"] + 1)

    cand_ssn_T = (
        sell_tminus2.merge(
            sell_tminus1,
            on=["cik", "reporting_cik", "month", "year"],
            how="inner"
        )
        [["cik", "reporting_cik", "month", "year"]]
        .drop_duplicates()
    )

    # Insider traded in T (used to filter out non-silence)
    any_T = g[g["has_any"]][["cik", "reporting_cik", "month", "year"]]

    # PPN: purchase-purchase-no trade
    ppn_T = (
        cand_ppn_T.merge(any_T, on=["cik", "reporting_cik", "month", "year"],
                         how="left", indicator=True)
                 .loc[lambda x: x["_merge"] == "left_only"]
                 .drop(columns="_merge")
    )

    # SSN: sell-sell-no trade
    ssn_T = (
        cand_ssn_T.merge(any_T, on=["cik", "reporting_cik", "month", "year"],
                         how="left", indicator=True)
                 .loc[lambda x: x["_merge"] == "left_only"]
                 .drop(columns="_merge")
    )

    # Convert to firm–month Period[M]
    def to_month(df_sub):
        out = df_sub.copy()
        out["month"] = pd.PeriodIndex(
            pd.to_datetime(dict(year=out["year"], month=out["month"], day=1)),
            freq="M"
        )
        return out[["cik", "month"]]

    ppn_e = to_month(ppn_T).assign(is_ppn=1.0)
    ssn_e = to_month(ssn_T).assign(is_ssn=1.0)

    # Aggregate to firm-month, convert to float32 for ML
    firm_m = (
        ppn_e.merge(ssn_e, on=["cik", "month"], how="outer")
             .fillna(0)
             .groupby(["cik", "month"], as_index=False)
             .max()
             .astype({"is_ppn": "float32", "is_ssn": "float32"})
    )

    return firm_m[["cik", "month", "is_ppn", "is_ssn"]]

### Call Function to Classify Insider Silence Events ###
insider_silence_monthly = classify_insider_silence_fast(insider_trades)

### Ensure month is datetime and end of month ###

# Set month to timestamp
insider_silence_monthly["month"] = insider_silence_monthly["month"].dt.to_timestamp("M")

# Ensure that period is within analysis window
insider_silence_monthly = insider_silence_monthly[
    (insider_silence_monthly["month"] >= pd.to_datetime(START_DATE)) &
    (insider_silence_monthly["month"] <= pd.to_datetime(END_DATE))
].reset_index(drop=True)

**Merging Features to Insider Trades**

In [ ]:
### Insider trades with features ###

# Create final insider trades dataframe copy
insider_trades_final = insider_trades.copy()

# Drop filing_date, and rename transaction_date to month
insider_trades_final = insider_trades_final.drop(columns=["filing_date"]).rename(columns={"transaction_date": "month"})

# Set month to month end timestamp
insider_trades_final["month"] = insider_trades_final["month"].dt.to_period("M").dt.to_timestamp("M")

# Merge with insider_trader_types_monthly to get trader types
insider_trades_final = insider_trades_final.merge(
    insider_trader_types_monthly,
    on=["cik", "month"],
    how="left"
)

# Create columns named is_count_purchase and is_count_sell to sum the number of purchase/sell transactions
insider_trades_final["is_count_purchase"] = insider_trades_final["is_txn_purchase"].astype("float32")
insider_trades_final["is_count_sell"] = insider_trades_final["is_txn_sell"].astype("float32")

# Aggregate to permno/transaction_month
insider_trades_final = (
    insider_trades_final
    .groupby(["cik", "month"], as_index=False)
    .agg({      
        # Permno, take last
        "permno": "last", 
        
        # Binary indicators, use max
        "is_txn_purchase": "max",
        "is_txn_sell": "max",
        
        "is_txn_purchase_x_is_tit_ceo": "max",
        "is_txn_purchase_x_is_tit_cfo": "max",
        "is_txn_purchase_x_is_tit_coo": "max",
        "is_txn_purchase_x_is_tit_director": "max",
        "is_txn_purchase_x_is_tit_other_officer": "max",
        "is_txn_purchase_x_is_tit_ten_percent_owner": "max",
        "is_txn_purchase_x_is_tit_vice_president": "max",
        
        "is_txn_sell_x_is_tit_ceo": "max",
        "is_txn_sell_x_is_tit_cfo": "max",
        "is_txn_sell_x_is_tit_coo": "max",
        "is_txn_sell_x_is_tit_director": "max",
        "is_txn_sell_x_is_tit_other_officer": "max",
        "is_txn_sell_x_is_tit_ten_percent_owner": "max",
        "is_txn_sell_x_is_tit_vice_president": "max",
        
        "is_opp_buy": "max",
        "is_opp_sell": "max",
        "is_rtn_buy": "max",
        "is_rtn_sell": "max",

        # Volume, use sum 
        "is_vol_purchase": "sum",
        "is_vol_sell": "sum",
        
        # Count, use sum 
        "is_count_purchase": "sum",
        "is_count_sell": "sum",
        "is_txn_purchase_cluster": "sum",
        "is_txn_sell_cluster": "sum",
    })
)

# # Display the first few rows of the aggregated dataframe
# insider_trades_final.head()


In [ ]:
### is_ppn and is_ssn ###

# Aggregate to cik/month
insider_silence_monthly = (
    insider_silence_monthly
    .groupby(["cik", "month"], as_index=False)
    .agg({      
        "is_ppn": "max",
        "is_ssn": "max"
    })
)

### Lag 1 month ###

# Lag month such that 2021-01 becomes 2021-02
insider_silence_monthly["month"] = insider_silence_monthly["month"] + pd.offsets.MonthEnd(1)

# Ensure that month falls between START_DATE and END_DATE
insider_silence_monthly = insider_silence_monthly[
    (insider_silence_monthly["month"] >= START_DATE) & (insider_silence_monthly["month"] <= END_DATE)
]

# Reset index
insider_silence_monthly = insider_silence_monthly.reset_index(drop=True)

In [ ]:
### Create NPR ###

# Create monthly DataFrame and sort by cik then month
insider_trades_final = insider_trades_final.sort_values(["cik", "month"])

# Construct rolling_window function
def rolling_window(df, col):
    out = []
    
    for idx, row in df.iterrows():
        # 6-month trailing window using month-end logic
        start = row["month"] + pd.offsets.MonthEnd(-6)
        
        # mask selecting all rows within the window
        mask = (df["month"] > start) & (df["month"] <= row["month"])
        
        # sum the column over the window
        out.append(df.loc[mask, col].sum())
        
    # return as a Series with the original index
    return pd.Series(out, index=df.index)

# Apply to each variable and each issuer
for col in ["is_count_purchase", "is_count_sell", "is_vol_purchase", "is_vol_sell"]:
    insider_trades_final[f"{col}_6m"] = (
        insider_trades_final.groupby("cik", group_keys=False)
               .apply(lambda x: rolling_window(x, col))
)

/var/folders/k3/9jyk10k50tg1bg0x762zj6200000gn/T/ipykernel_41890/4188858623.py:27: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: rolling_window(x, col))
/var/folders/k3/9jyk10k50tg1bg0x762zj6200000gn/T/ipykernel_41890/4188858623.py:27: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: rolling_window(x, col))
/var/folders/k3/9jyk10k50tg1bg0x762zj6200000gn/T/ipykernel_41890/4188858623.py:2

In [ ]:
def saferatio(num, den):
    # Create output Series initialized to 0.0, with same index as num
    out = pd.Series(0.0, index=num.index)

    # mask where denominator is NOT zero
    mask = den != 0

    # compute ratio only where safe (den != 0)
    out[mask] = num[mask] / den[mask]
    
    return out

# NPR calculations
insider_trades_final["is_npr_count"] = saferatio(
        insider_trades_final["is_count_purchase_6m"] - insider_trades_final["is_count_sell_6m"],
        insider_trades_final["is_count_purchase_6m"] + insider_trades_final["is_count_sell_6m"],
    )

insider_trades_final["is_npr_volume"] = saferatio(
        insider_trades_final["is_vol_purchase_6m"] - insider_trades_final["is_vol_sell_6m"],
        insider_trades_final["is_vol_purchase_6m"] + insider_trades_final["is_vol_sell_6m"],
    )

In [ ]:
### Net Clustering ###
insider_trades_final["is_net_cluster"] = (
    insider_trades_final["is_txn_purchase_cluster"] - insider_trades_final["is_txn_sell_cluster"]
).astype("float32")

**Extract features**

**Lags and dropping unnecessary columns** 

Relies on monthly data. 

In [ ]:
### Lag 1 month ###

# Lag month such that 2021-01 becomes 2021-02
insider_trades_final["month"] = insider_trades_final["month"] + pd.offsets.MonthEnd(1)

# Ensure that month falls between START_DATE and END_DATE
insider_trades_final = insider_trades_final[
    (insider_trades_final["month"] >= START_DATE) & (insider_trades_final["month"] <= END_DATE)
]

# Reset index
insider_trades_final = insider_trades_final.reset_index(drop=True)

# # Display the first few rows of the final dataframe
# insider_trades_final.head()

In [ ]:
### Dropping features that are not used anymore ###
insider_trades_final = insider_trades_final.drop(columns=["is_txn_purchase", "is_txn_sell", "is_vol_purchase", 
                                                          "is_vol_sell", "is_count_purchase", "is_count_sell",
                                                          "is_count_purchase_6m", "is_count_sell_6m",
                                                          "is_vol_purchase_6m", "is_vol_sell_6m", 
                                                          "is_txn_purchase_cluster", "is_txn_sell_cluster"]
)

**Merge insider trades with GKX data**

In [ ]:
### Merge all but is_ppn and is_ssn ### 

# Ensure datetime normalized and month end for both month columns in both dataframes
merged_df["month"] = pd.to_datetime(merged_df["month"]).dt.normalize()
insider_trades_final["month"] = pd.to_datetime(insider_trades_final["month"]).dt.normalize()

# Merge
merged_df_final = pd.merge(
    merged_df,
    insider_trades_final,
    on=["permno", "cik", "month"],
    how="left"
)

# Create an assertion 
assert len(merged_df_final) == len(merged_df), "Merged DataFrame length mismatch!"

In [ ]:
### Merge is_ppn and is_ssn ###

# Ensure datetime normalized and month end for both month columns in both dataframes
merged_df_final["month"] = pd.to_datetime(merged_df_final["month"]).dt.normalize()
insider_silence_monthly["month"] = pd.to_datetime(insider_silence_monthly["month"]).dt.normalize()

# Merge
merged_df_final = pd.merge(
    merged_df_final,
    insider_silence_monthly,
    on=["cik", "month"],
    how="left"
)

# Create an assertion 
assert len(merged_df_final) == len(merged_df), "Merged DataFrame length mismatch!"

**Rank transformation of stock characteristics**

In [ ]:
# Fill na for dummies
dummy_cols = [
    "is_opp_buy", "is_opp_sell", 
    "is_rtn_buy", "is_rtn_sell", 
    "is_ppn", "is_ssn",
    
    "is_txn_purchase_x_is_tit_ceo",
    "is_txn_purchase_x_is_tit_cfo",
    "is_txn_purchase_x_is_tit_coo",
    "is_txn_purchase_x_is_tit_director",
    "is_txn_purchase_x_is_tit_other_officer",
    "is_txn_purchase_x_is_tit_ten_percent_owner",
    "is_txn_purchase_x_is_tit_vice_president",
    
    "is_txn_sell_x_is_tit_ceo",
    "is_txn_sell_x_is_tit_cfo",
    "is_txn_sell_x_is_tit_coo",
    "is_txn_sell_x_is_tit_director",
    "is_txn_sell_x_is_tit_other_officer",
    "is_txn_sell_x_is_tit_ten_percent_owner",
    "is_txn_sell_x_is_tit_vice_president",
]

# Rename dummies
for col in dummy_cols:
    if col in merged_df_final.columns:
        merged_df_final[col] = merged_df_final[col].fillna(0).astype("float32")
        
# Assert that dummy cols does not have any missing values
for col in dummy_cols:
    if col in merged_df_final.columns:
        assert merged_df_final[col].isna().sum() == 0, f"Missing values found in column {col}"

Rank transform stock characteristics

In [ ]:
### Rank transform ###

# Function for ranking
def rank_transform(x):
    """
    Rank transform function equivalent to the R version.
    Scales values to range from -1 to 1.
    """
    # Get ranks
    rank_x = x.rank(axis=0, method='average', na_option='keep') # R's "average" method is equivalent to pandas' "average"; keep ensures that NA value are not ranked

    # Count non-NA values
    max_rank = x.notna().sum() # Counts number of non-NA values
    min_rank = 1

    if max_rank <= 1:
        return pd.Series(np.nan, index=x.index) # All values are NA
    else:
        return 2 * (((rank_x - min_rank) / (max_rank - min_rank)) - 0.5) # Scale to [-1, 1]

# List of characteristic columns
char_cols = [col for col in merged_df_final.columns if col.startswith("char_")] + [col for col in merged_df_final.columns if (col.startswith("is_npr_") or col.startswith("is_net_cluster"))]

# Apply rank transformation by month
merged_df_final[char_cols] = merged_df_final.groupby("month")[char_cols].transform(rank_transform)

# # Display the first couple of rows
# display(merged_df_final.head())

Two-step procedure: 
1. Replace missing values with cross-sectional median. 
2. If cross-sectional median is undefined → replace with zero which for large and diverse datasets goes to zero. 

In [ ]:
### Handle missing values ### 

# Select characteristic columns
char_cols = [col for col in merged_df_final.columns if col.startswith("char_")] + [col for col in merged_df_final.columns if (col.startswith("is_npr_") or col.startswith("is_net_cluster"))]

# Step 1: fill NAs with within-month median
merged_df_final[char_cols] = (
    merged_df_final.groupby("month")[char_cols]
    .transform(lambda x: x.fillna(x.median() if x.notna().any() else np.nan))
)

# Step 2: replace any leftover NAs with 0
merged_df_final[char_cols] = merged_df_final[char_cols].fillna(0)

# # Display the first couple of rows
# display(merged_df_final.head())

**Create interaction terms and one-hot encode industry classification**

Create interaction terms between macro predictors and firm characteristics. 

Note that the macro predictors are not standardized:

If you choose to standardize the macro predictors before creating interaction terms, this must be done using the training data only to avoid look-ahead bias.

In [ ]:
### Create interaction terms ###

# Identify characteristic and macro columns
char_cols = [col for col in merged_df_final.columns if col.startswith("char_")] + [col for col in merged_df_final.columns if (col.startswith("is_npr_") or col.startswith("is_net_cluster"))]
macro_cols = [col for col in merged_df_final.columns if col.startswith("macro_")]

# Create interaction terms using dictionary comprehension
interaction_terms = {
    f"inter_{char}_x_{macro}": merged_df_final[char].values * merged_df_final[macro].values
    for char in char_cols
    for macro in macro_cols
}

# Turn interaction terms into a DataFrame
interaction_df = pd.DataFrame(interaction_terms, index=merged_df_final.index)

# Concat interaction terms to the original DataFrame
merged_df_final = pd.concat([merged_df_final, interaction_df], axis=1)

# # Drop macro predictors as they are now represented in the interaction terms
# merged_df_final = merged_df_final.drop(columns=macro_cols)

# # Display the first couple of rows
# display(merged_df_final.head())

One-hot encode industry classification

In [ ]:
### One-hot encode sic2 using float32 ###

# Downcast if pd allows it
merged_df_final = convert_formats(merged_df_final)

# One-hot encode sic2 using float32
merged_df_final = pd.get_dummies(
    merged_df_final,
    columns=["sic2"],
    prefix="sic2",           # Prefix to seperate from dummies not adhering to insider trading
    drop_first=True,         # Drop first to avoid dummy variable trap - the dummies are mutually exclusive
    dtype="float32",
    sparse=False             # Use dense format to save as parquet file
)

# # Display the first couple of rows
# display(merged_df_final.head())

In [ ]:
### Set insider trading variables to last ###

# Get columns: other and insider
insider_cols = [col for col in merged_df_final.columns if col.startswith("is_") or col.startswith("inter_is_")]
other_cols = [col for col in merged_df_final.columns if col not in insider_cols]

# Reorder columns
final_cols = other_cols + insider_cols

# Final DataFrame with reordered columns
merged_df_final = merged_df_final[final_cols]

**Dropping columns with p > 0.05 from DML test**

The columns are not dropped earlier to ensure that all variables are available if needed. 

In [ ]:
### Drop insignificant columns from DML ###

# Columns to drop
cols_to_drop = [
    "is_txn_sell_x_is_tit_vice_president",
    "is_txn_purchase_x_is_tit_ten_percent_owner",
    "is_ppn",
    "is_ssn",
    "is_txn_sell_x_is_tit_cfo",
    "is_txn_sell_x_is_tit_director",
    "is_txn_sell_x_is_tit_other_officer",
    "is_txn_sell_x_is_tit_coo",
] + [col for col in merged_df_final.columns if "npr_count" in col]

# Drop columns if they exist
existing = [col for col in cols_to_drop if col in merged_df_final.columns]
merged_df_final = merged_df_final.drop(columns=existing)

# # Display the first couple of rows
# display(merged_df_final.head())

**Save final**

In [ ]:
### Assert that len after dropping columns is the same as not dropping ###
assert len(merged_df_final) == len(merged_df_final.drop_duplicates())

In [ ]:
### Save col_list for later use ###

insider_cols = [col for col in merged_df_final.columns 
                if col.startswith("is_") or col.startswith("inter_is_")]

gkx_cols = [col for col in merged_df_final.columns 
            if col.startswith("char_")
            or col.startswith("sic2_") 
            or (col.startswith("inter_") and not col.startswith("inter_is_"))]

info_cols = [col for col in merged_df_final.columns 
             if col not in insider_cols + gkx_cols]


# Base path for output files
base_path = "/Users/jonas/Library/CloudStorage/OneDrive-UniversityofCopenhagen/Kandidat/Thesis/2.0 Data/Characteristics Prepared/With Outsider/"


# Save insider_cols
with open(base_path + "insider_cols.txt", "w") as f:
    for col in insider_cols:
        f.write(f"{col}\n")

# Save gkx_cols
with open(base_path + "gkx_cols.txt", "w") as f:
    for col in gkx_cols:
        f.write(f"{col}\n")

# Save info_cols
with open(base_path + "info_cols.txt", "w") as f:
    for col in info_cols:
        f.write(f"{col}\n")

In [ ]:
### Save prepared data ###

# Path
path = "/Users/jonas/Library/CloudStorage/OneDrive-UniversityofCopenhagen/Kandidat/Thesis/2.0 Data/Characteristics Prepared/With Outsider/with_outsider.parquet" # Update this path to your desired output folder

# Save to parquet
merged_df_final.to_parquet(
    path,
    engine="pyarrow",
    index=False
)

For yearly datasets - if needed:

In [ ]:
# ### Save prepared data ###

# # Base output path
# BASE_PATH = Path(
#     "/Users/jonas/Library/CloudStorage/OneDrive-UniversityofCopenhagen/Kandidat/Thesis/2.0 Data/Characteristics Prepared" # Update this path to your desired output folder
# )

# # Extract year from 'month'
# merged_df_final["year"] = pd.to_datetime(merged_df_final["month"]).dt.year

# # Save as partitioned Parquet dataset
# path = BASE_PATH / "final_partitioned"
# merged_df_final.to_parquet(
#     path,
#     partition_cols=["year"],
#     engine="pyarrow",
#     index=False
# )